In [ ]:
# AI Text Detection Model Training Script - Final Fixed Version
# For Google Colab with T4 GPU - Authentiscore Project

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset as TorchDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import evaluate
import random

# Set paths - update these paths as needed
DATA_PATH = '/content/drive/MyDrive/AI_Human.csv'  # Update this path
OUTPUT_DIR = 'ml_models'  
MODEL_NAME = 'distilbert-base-uncased' 

# Create output directory if not exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

print("Loading dataset...")
try:
    df = pd.read_csv(
        
    )
    print(f"Original dataset shape: {df.shape}")
    print(f"Column names: {df.columns.tolist()}")

    # Check if the necessary columns exist
    if 'text' not in df.columns:
        raise ValueError("Dataset must have a 'text' column")

    # Rename 'generated' column to 'label' if it exists
    if 'generated' in df.columns:
        # Ensure the values are proper integers (0 and 1)
        df['generated'] = df['generated'].astype(float).astype(int)
        df = df.rename(columns={'generated': 'label'})
        print("Renamed 'generated' column to 'label'")
    elif 'is_ai_generated' in df.columns:
        df = df.rename(columns={'is_ai_generated': 'label'})
        print("Renamed 'is_ai_generated' column to 'label'")

    # Check if 'label' column exists after renaming
    if 'label' not in df.columns:
        raise ValueError("Could not find a 'label' column in the dataset")

    # Make sure label column contains only 0 and 1 integers
    df['label'] = df['label'].astype(int)
    unique_labels = df['label'].unique()
    print(f"Unique values in label column: {unique_labels}")

    # Balance the dataset
    label_counts = df['label'].value_counts()
    print(f"Label distribution: {label_counts.to_dict()}")

    # Limit to 1000 samples total (500 for each class)
    samples_per_class = min(500, min(label_counts[0], label_counts[1]))

    df_class_0 = df[df['label'] == 0].sample(samples_per_class, random_state=42)
    df_class_1 = df[df['label'] == 1].sample(samples_per_class, random_state=42)

    df = pd.concat([df_class_0, df_class_1], ignore_index=True)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle the data

    print(f"Reduced dataset to {len(df)} balanced samples")
    print(f"New label distribution: {df['label'].value_counts().to_dict()}")

except Exception as e:
    print(f"Error loading or processing dataset: {e}")
    raise

# Split dataset into train and validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
print(f"Training set: {len(train_df)} samples")
print(f"Validation set: {len(val_df)} samples")

# Prepare tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Set a smaller max_length for T4 GPU
MAX_LENGTH = 256  # Reduced from 512 to save memory

# Custom dataset class that correctly formats the labels
class TextClassificationDataset(TorchDataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])  # Make sure 'labels' (not 'label') is used
        return item

    def __len__(self):
        return len(self.labels)

# Prepare datasets using the custom dataset class
train_dataset = TextClassificationDataset(
    train_df['text'].tolist(),
    train_df['label'].tolist(),
    tokenizer,
    MAX_LENGTH
)

val_dataset = TextClassificationDataset(
    val_df['text'].tolist(),
    val_df['label'].tolist(),
    tokenizer,
    MAX_LENGTH
)

print("Datasets prepared successfully")

# Load pre-trained model for sequence classification
print(f"Loading model: {MODEL_NAME}")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
)

# Define compute_metrics function
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")

    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"],
        "precision": precision["precision"],
        "recall": recall["recall"]
    }

# Define training arguments - optimized for T4 GPU
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,  # Reduced for T4 GPU
    per_device_eval_batch_size=8,   # Reduced for T4 GPU
    num_train_epochs=3,             # Using fewer epochs for faster training
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    fp16=True,                      # Enable mixed precision training for T4 GPU
    logging_steps=10,
    save_total_limit=2,             # Keep only the 2 best checkpoints
    disable_tqdm=False,             # Show progress bar
    # Disable wandb reporting to simplify things
    report_to="none",
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train model
print("Starting model training...")
try:
    trainer.train()

    # Evaluate model
    print("Evaluating model...")
    eval_results = trainer.evaluate()
    print(f"Evaluation results: {eval_results}")

    # Save model and tokenizer
    print("Saving model and tokenizer...")
    trainer.save_model(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print(f"Model saved to {OUTPUT_DIR}")

    # Test model with sample text
    print("Testing model with sample texts...")
    from transformers import pipeline

    classifier = pipeline("text-classification", model=OUTPUT_DIR, tokenizer=OUTPUT_DIR)

    test_texts = [
        "This is a human-written test sentence to check model performance.",
        "The quantum mechanics underlying photosynthesis have been extensively studied."
    ]

    for text in test_texts:
        result = classifier(text)[0]
        label = "AI-generated" if result["label"] == "LABEL_1" else "Human-written"
        print(f"Text: {text}")
        print(f"Prediction: {label} (confidence: {result['score']:.4f})")
        print("---")

    print("Training and evaluation complete!")

except Exception as e:
    print(f"Error during training or evaluation: {e}")
    import traceback
    traceback.print_exc()
    # Save model anyway in case of partial training
    try:
        trainer.save_model(OUTPUT_DIR + "_partial")
        tokenizer.save_pretrained(OUTPUT_DIR + "_partial")
        print(f"Partially trained model saved to {OUTPUT_DIR}_partial")
    except:
        print("Could not save partial model")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading dataset...
Original dataset shape: (487235, 2)
Column names: ['text', 'generated']
Renamed 'generated' column to 'label'
Unique values in label column: [0 1]
Label distribution: {0: 305797, 1: 181438}
Reduced dataset to 1000 balanced samples
New label distribution: {1: 500, 0: 500}
Training set: 800 samples
Validation set: 200 samples
Datasets prepared successfully
Loading model: distilbert-base-uncased


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting model training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.204200,0.219057,0.925000,0.924576,0.934783,0.925000
2,0.134100,0.057751,0.980000,0.979998,0.980192,0.980000
3,0.065700,0.091480,0.975000,0.974994,0.975428,0.975000


Evaluating model...


Evaluation results: {'eval_loss': 0.057750824838876724, 'eval_accuracy': 0.98, 'eval_f1': 0.97999799979998, 'eval_precision': 0.9801920768307323, 'eval_recall': 0.98, 'eval_runtime': 0.5376, 'eval_samples_per_second': 372.023, 'eval_steps_per_second': 46.503, 'epoch': 3.0}
Saving model and tokenizer...
Model saved to /content/drive/MyDrive/authentiscore_model
Testing model with sample texts...


Device set to use cuda:0


Text: This is a human-written test sentence to check model performance.
Prediction: AI-generated (confidence: 0.9540)
---
Text: The quantum mechanics underlying photosynthesis have been extensively studied.
Prediction: AI-generated (confidence: 0.9795)
---
Training and evaluation complete!


In [ ]:
# Simple test script for Authentiscore Model
# Run this after training to test your model on new texts

from transformers import pipeline

# Load your model
MODEL_PATH = '/content/drive/MyDrive/authentiscore_modelb'

# Create the classifier pipeline
try:
    classifier = pipeline("text-classification", model=MODEL_PATH, tokenizer=MODEL_PATH)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Trying to load partial model instead...")
    try:
        classifier = pipeline("text-classification",
                             model=MODEL_PATH + "_partial",
                             tokenizer=MODEL_PATH + "_partial")
        print("Partial model loaded successfully!")
    except Exception as e:
        print(f"Error loading partial model: {e}")
        raise

# Test texts (add your own examples)
test_texts = [
    "This is a human-written text about my day at the park. I saw birds and enjoyed the sunshine.",
    "The implementation of neural network architectures facilitates the optimization of computational resources while maintaining performance metrics within acceptable parameters.",
    "My grandmother's recipe for apple pie includes cinnamon, sugar, and a secret ingredient she never revealed to anyone.",
    "Utilizing advanced algorithms and machine learning techniques, the system processes large volumes of data to extract meaningful patterns and insights.",
    "I couldn't believe how the movie ended! The plot twist was completely unexpected and left me speechless.",
    "The quantitative analysis demonstrates a statistically significant correlation between the variables, suggesting a causal relationship worthy of further investigation."
]

# Process each text and display results
print("\n=== AUTHENTISCORE DETECTION RESULTS ===\n")
for i, text in enumerate(test_texts):
    result = classifier(text)[0]
    confidence = result['score'] * 100

    if result["label"] == "LABEL_1":
        detection = "AI-GENERATED"
        emoji = "🤖"
    else:
        detection = "HUMAN-WRITTEN"
        emoji = "👤"

    print(f"Text #{i+1}: {text[:50]}..." if len(text) > 50 else f"Text #{i+1}: {text}")
    print(f"{emoji} {detection} (Confidence: {confidence:.1f}%)")
    print("-" * 50)

Device set to use cuda:0


Model loaded successfully!

=== AUTHENTISCORE DETECTION RESULTS ===

Text #1: This is a human-written text about my day at the p...
🤖 AI-GENERATED (Confidence: 95.3%)
--------------------------------------------------
Text #2: The implementation of neural network architectures...
🤖 AI-GENERATED (Confidence: 99.3%)
--------------------------------------------------
Text #3: My grandmother's recipe for apple pie includes cin...
🤖 AI-GENERATED (Confidence: 96.4%)
--------------------------------------------------
Text #4: Utilizing advanced algorithms and machine learning...
🤖 AI-GENERATED (Confidence: 99.2%)
--------------------------------------------------
Text #5: I couldn't believe how the movie ended! The plot t...
🤖 AI-GENERATED (Confidence: 94.7%)
--------------------------------------------------
Text #6: The quantitative analysis demonstrates a statistic...
🤖 AI-GENERATED (Confidence: 99.3%)
--------------------------------------------------


In [ ]:
# Convert PyTorch model to ONNX format for lightweight deployment
# For Google Colab

# First, install required packages
!pip install onnx onnxruntime

# Import system libraries
import os
import sys
import torch
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
print("Google Drive mounted successfully")

# Verify ONNX installation
try:
    import onnx
    print(f"ONNX version: {onnx.__version__}")
    import onnxruntime
    print(f"ONNX Runtime version: {onnxruntime.__version__}")
except ImportError as e:
    print(f"Error importing ONNX: {e}")
    print("Please restart the runtime after package installation and run this script again.")
    sys.exit(1)

# Import transformer libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Set model path
model_path = '/content/drive/MyDrive/authentiscore_model'
print(f"Loading model from: {model_path}")

# Verify model path exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model directory not found at {model_path}. Please check the path.")

try:
    # Load the trained model and tokenizer
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    print("Loading model...")
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    model.eval()  # Set model to evaluation mode

    print("Model and tokenizer loaded successfully")

    # Create dummy input for tracing
    print("Creating dummy input for ONNX export...")
    dummy_input = tokenizer("This is a sample text", return_tensors="pt",
                            truncation=True, padding="max_length", max_length=256)

    # Define export path
    onnx_path = os.path.join(model_path, "model.onnx")

    # Export to ONNX format
    print("Exporting model to ONNX format with opset version 14...")
    torch.onnx.export(
        model,
        (dummy_input.input_ids, dummy_input.attention_mask),
        onnx_path,
        input_names=["input_ids", "attention_mask"],
        output_names=["logits"],
        dynamic_axes={
            "input_ids": {0: "batch_size"},
            "attention_mask": {0: "batch_size"},
            "logits": {0: "batch_size"}
        },
        opset_version=14
    )

    # Save tokenizer files to a specific directory
    tokenizer_path = os.path.join(model_path, "tokenizer")
    print(f"Saving tokenizer files to: {tokenizer_path}")
    tokenizer.save_pretrained(tokenizer_path)

    # Verify the ONNX model
    print("Verifying ONNX model...")
    onnx_model = onnx.load(onnx_path)
    onnx.checker.check_model(onnx_model)
    print("ONNX model verification successful")

    print("Conversion complete!")
    print(f"Model exported to ONNX format and saved at: {onnx_path}")
    print(f"Tokenizer files saved at: {tokenizer_path}")

    # Verify exported model file size
    onnx_size_mb = os.path.getsize(onnx_path) / (1024 * 1024)
    print(f"ONNX model file size: {onnx_size_mb:.2f} MB")

    print("\nNext steps:")
    print("1. Download the model.onnx file and tokenizer folder from your Google Drive")
    print("2. Use them with the ONNX Runtime FastAPI application locally")

except Exception as e:
    print(f"Error during model conversion: {e}")
    import traceback
    traceback.print_exc()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.6 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully
ONNX version: 1.17.0
ONNX Runtime version: 1.21.0
Loading model from: /content/drive/MyDrive/authentiscore_model
Loading tokenizer...
Loading model...
Model and tokenizer loaded successfully
Creating dummy input for ONNX export...
Exporting model to ONNX format with opset version 14...
Saving tokenizer files to: /content/drive/MyDrive/authentiscore_model/tokenizer
Verifying ONNX model...
ONNX model verification successful
Conversion complete!
Model exported to ONNX format and saved at: /content/drive/MyDrive/authent